In [25]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [26]:
data = [[110, 120, 130, 110, 115],
        [130, 130, 110,  90, 115],
        [110, 140, 130, 100,  95],
        [120, 110, 120, 120, 125],
        [100, 120, 150, 110, 105],
        [ 90, 100, 140,  80, 135]]

VEG = ['VEG1', 'VEG2']
NVEG = ['OIL1', 'OIL2', 'OIL3']
OILS = VEG + NVEG
MONTHS = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN']
PREV_MONTH = {'JAN': 'INITIAL', 
              'FEB': 'JAN',
              'MAR': 'FEB',
              'APR': 'MAR',
              'MAY': 'APR',
              'JUN': 'MAY'}

costs = pd.DataFrame(data, columns=OILS, index=MONTHS)
hardness = dict(zip(OILS, [8.8, 6.1, 2.0, 4.2, 5.0]))

product_price  = 150
storage_cost   = 5
storage_limit  = 1000
veg_ref_limit  = 200
nveg_ref_limit = 250
upper_hardness = 6
lower_hardness = 3

In [27]:
model2 = gp.Model('Food Manufacture II')

# add vars
buy = model2.addVars(OILS, MONTHS,
                     name='buy')
use = model2.addVars(OILS, MONTHS,
                     name='use')
bin_use = model2.addVars(OILS, MONTHS,
                         name='bin_use',
                         vtype=gp.GRB.BINARY)    
produce = model2.addVars(MONTHS,
                         name='produce')
store = model2.addVars(OILS, MONTHS+['INITIAL'],
                       name='store')

# set upper and lower bounds of "store" variables
for i in OILS:
    store[i,'INITIAL'].setAttr('ub', 500.0)
    store[i,'INITIAL'].setAttr('lb', 500.0)
    store[i,'JUN'].setAttr('ub', 500.0)
    store[i,'JUN'].setAttr('lb', 500.0)

# objective function
model2.setObjective(gp.quicksum(product_price*produce[t] - 
                                gp.quicksum(costs[i][t]*buy[i,t]+storage_cost*store[i,t] for i in OILS) for t in MONTHS),
                    GRB.MAXIMIZE)

# add constraints
model2.addConstrs((gp.quicksum(use[i,t] for i in VEG) <= veg_ref_limit for t in MONTHS),
                  name='refine_veg')
model2.addConstrs((gp.quicksum(use[i,t] for i in NVEG) <= nveg_ref_limit for t in MONTHS),
                  name='refine_nveg')
model2.addConstrs((gp.quicksum(use[i,t] for i in OILS) - produce[t] == 0 for t in MONTHS),
                  name='mass_conservation')
model2.addConstrs((gp.quicksum(hardness[i]*use[i,t] for i in OILS) - upper_hardness*produce[t] <= 0 for t in MONTHS),
                  name='upper_hardness')
model2.addConstrs((gp.quicksum(hardness[i]*use[i,t] for i in OILS) - lower_hardness*produce[t] >= 0 for t in MONTHS),
                  name='lower_hardness')
model2.addConstrs((buy[i,t] + store[i,PREV_MONTH[t]] - use[i,t] - store[i,t] == 0 for i in OILS for t in MONTHS),
                  name='storage_link')
model2.addConstrs((store[i,t] <= 1000 for i in OILS for t in MONTHS),
                  name='storage_limit')
model2.addConstrs((gp.quicksum(bin_use[i,t] for i in OILS) <= 3 for t in MONTHS),
                  name='condition_1')
model2.addConstrs((use[i,t] - 200*bin_use[i,t] <= 0 for i in VEG for t in MONTHS),       # M=200 for vegetable oils
                  name='condition_2_max')
model2.addConstrs((use[i,t] - 250*bin_use[i,t] <= 0 for i in NVEG for t in MONTHS),      # M=250 for non vegetable oils
                  name='condition_2_max')
model2.addConstrs((use[i,t] - 20*bin_use[i,t] >= 0 for i in OILS for t in MONTHS),
                  name='condition_2_min')
model2.addConstrs((bin_use['VEG1',t] - bin_use['OIL3',t] <= 0 for t in MONTHS),
                  name='condition_3_veg1')
model2.addConstrs((bin_use['VEG2',t] - bin_use['OIL3',t] <= 0 for t in MONTHS),
                  name='condition_3_veg2')
model2.update()

In [28]:
model2.write('Food Manufacture II.lp')
model2.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 131 columns and 462 nonzeros
Model fingerprint: 0xfdb977e0
Variable types: 101 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 5e+02]
  RHS range        [3e+00, 1e+03]
Found heuristic solution: objective -75000.00000
Presolve removed 40 rows and 50 columns
Presolve time: 0.01s
Presolved: 128 rows, 81 columns, 360 nonzeros
Variable types: 51 continuous, 30 integer (30 binary)

Root relaxation: objective 1.071833e+05, 71 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 107183.333    0    9 -75000.000 107183.333   243%     -    0s
H    0     0                    97765.740741 107183.333 

In [29]:
print('Profit of ${:.2f}'.format(model2.objVal))
produce_solution = pd.DataFrame([], columns=['PRODUCE'], index=MONTHS)
produce_solution = produce_solution.fillna(0)
for t in produce.keys():
    produce_solution['PRODUCE'][t] = produce[t].x
print('===PRODUCTION PLAN')
print(produce_solution)

buy_solution = pd.DataFrame([], columns=OILS, index=MONTHS)
buy_solution = buy_solution.fillna(0)
for i, t in buy.keys():
    buy_solution[i][t] = buy[i,t].x
print('===BUYING PLAN')
print(buy_solution)

use_solution = pd.DataFrame([], columns=OILS, index=MONTHS)
use_solution = use_solution.fillna(0)
for i, t in use.keys():
    use_solution[i][t] = use[i,t].x
print('===REFINING PLAN')
print(use_solution)

store_solution = pd.DataFrame([], columns=OILS, index=['INITIAL']+MONTHS)
store_solution = store_solution.fillna(0)
for i, t in store.keys():
    store_solution[i][t] = store[i,t].x
print('===STORAGE PLAN')
print(store_solution[1:])

Profit of $100278.70
===PRODUCTION PLAN
     PRODUCE
JAN      450
FEB      450
MAR      450
APR      404
MAY      405
JUN      450
===BUYING PLAN
     VEG1  VEG2  OIL1  OIL2  OIL3
JAN     0     0     0     0     0
FEB     0     0     0   190     0
MAR     0     0     0     0    39
APR     0     0     0     0     0
MAY     0     0     0     0   539
JUN   480   629     0   730     0
===REFINING PLAN
     VEG1  VEG2  OIL1  OIL2  OIL3
JAN    85   114     0     0   250
FEB    85   114     0     0   249
MAR     0   200     0   230    20
APR   154     0     0   230    20
MAY   155     0     0   230    19
JUN     0   199     0   230    19
===STORAGE PLAN
     VEG1  VEG2  OIL1  OIL2  OIL3
JAN   414   385   500   500   250
FEB   329   270   500   690     0
MAR   329    70   500   460    20
APR   174    70   500   230     0
MAY    19    70   500     0   520
JUN   500   500   500   500   500
